In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix

# ▶︎ Bases a processar
arquivos_dataset = [
    'VGG16_256_AVG.csv',  'VGG16_256_MAX.csv',
    'VGG19_128_AVG.csv',  'VGG19_128_MAX.csv',
    'VGG19_256_AVG.csv',  'VGG19_256_MAX.csv',
    'VGG16_256_AVG_PCA.csv', 'VGG16_256_MAX_PCA.csv',
    'VGG19_128_AVG_PCA.csv', 'VGG19_128_MAX_PCA.csv',
    'VGG19_256_AVG_PCA.csv', 'VGG19_256_MAX_PCA.csv'
]

tipos_numericos = ['int32','int64','float16','float32','float64']

# Classificadores Naive Bayes
classificadores = {
    'GaussianNB'    : GaussianNB(),
    'MultinomialNB' : MultinomialNB(),
    'ComplementNB'  : ComplementNB()
}

resultados = []
mc_dict = {}

kf = KFold(n_splits=10, shuffle=True, random_state=1)

for arquivo in arquivos_dataset:
    try:
        df = pd.read_csv(arquivo, encoding='utf-8')
    except FileNotFoundError:
        print(f'⚠️  Arquivo não encontrado: {arquivo}')
        continue

    if 'classe' not in df.columns:
        print(f'⚠️  Coluna "classe" ausente em {arquivo}')
        continue

    num_cols = list(df.select_dtypes(include=tipos_numericos).columns)
    if not num_cols:
        print(f'⚠️  Nenhum atributo numérico em {arquivo}')
        continue

    X_full = df[num_cols].copy()
    y = df['classe']

    for nome, clf in classificadores.items():
        # Garantir valores ≥ 0 se necessário
        if nome != 'GaussianNB':
            X_mod = X_full.copy()
            min_val = X_mod.min().min()
            if min_val < 0:
                shift = abs(min_val)
                X_mod += shift
        else:
            X_mod = X_full

        # Acurácia via cross_val_score
        scores = cross_val_score(clf, X_mod, y, scoring='accuracy', cv=kf)
        media = round(scores.mean() * 100, 2)
        desvio = round(scores.std() * 100, 2)

        # Matriz de confusão via cross_val_predict
        y_pred = cross_val_predict(clf, X_mod, y, cv=kf)
        cm = confusion_matrix(y, y_pred)
        mc_dict[(arquivo, nome)] = cm

        resultados.append({
            'Classificador' : nome,
            'Base'          : arquivo,
            'Acuracia_%'    : media,
            'Desvio_%'      : desvio
        })

        print(f'{nome:<14} | {arquivo:<30} → {media:.2f}% ± {desvio:.2f}%')

# ▶︎ Salvar CSV ordenado
df_res = pd.DataFrame(resultados)
df_res = df_res.sort_values(by=['Classificador', 'Base'])
df_res.to_csv('resultados_NaiveBayes_kfold.csv', index=False)
print('\n✅ CSV salvo como resultados_NaiveBayes_kfold.csv')

# ▶︎ Salvar matrizes de confusão
with open('matrizesConfusao_NB_kfold.txt', 'w') as f:
    for (base, nome), cm in mc_dict.items():
        f.write(f'\nBase: {base} | {nome}\n')
        f.write(np.array2string(cm))
        f.write('\n')
print('✅ Matrizes de confusão salvas em matrizesConfusao_NB_kfold.txt')


GaussianNB     | VGG16_256_AVG.csv              → 95.12% ± 1.89%
MultinomialNB  | VGG16_256_AVG.csv              → 97.50% ± 2.09%
ComplementNB   | VGG16_256_AVG.csv              → 97.38% ± 1.97%
GaussianNB     | VGG16_256_MAX.csv              → 97.25% ± 2.22%
MultinomialNB  | VGG16_256_MAX.csv              → 98.12% ± 1.51%
ComplementNB   | VGG16_256_MAX.csv              → 97.75% ± 1.56%
GaussianNB     | VGG19_128_AVG.csv              → 91.25% ± 1.48%
MultinomialNB  | VGG19_128_AVG.csv              → 94.75% ± 2.89%
ComplementNB   | VGG19_128_AVG.csv              → 92.88% ± 2.50%
GaussianNB     | VGG19_128_MAX.csv              → 94.00% ± 2.22%
MultinomialNB  | VGG19_128_MAX.csv              → 94.88% ± 2.34%
ComplementNB   | VGG19_128_MAX.csv              → 92.50% ± 2.90%
GaussianNB     | VGG19_256_AVG.csv              → 95.75% ± 1.70%
MultinomialNB  | VGG19_256_AVG.csv              → 98.00% ± 1.87%
ComplementNB   | VGG19_256_AVG.csv              → 97.25% ± 2.29%
GaussianNB     | VGG19_25

kfold

In [ ]:
import numpy as np
import pandas as pd

# ▶︎ Bases a processar
arquivos_dataset = [
    'VGG16_256_AVG.csv',
    'VGG16_256_MAX.csv',
    'VGG19_128_AVG.csv',
    'VGG19_128_MAX.csv',
    'VGG19_256_AVG.csv',
    'VGG19_256_MAX.csv',
    'VGG16_256_AVG_PCA.csv',
    'VGG16_256_MAX_PCA.csv',
    'VGG19_128_AVG_PCA.csv',
    'VGG19_128_MAX_PCA.csv',
    'VGG19_256_AVG_PCA.csv',
    'VGG19_256_MAX_PCA.csv'
]

tipos_numericos = ['int32', 'int64', 'float16', 'float32', 'float64']

# Lista para armazenar os resultados de cada base
resultados_kfold = []

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold, cross_val_score
# We don't need train_test_split, metrics, confusion_matrix, cross_val_predict for this version

# Loop sobre cada arquivo na lista
for arquivo in arquivos_dataset:
    print(f"Processando a base: {arquivo}")

    # Carregar o dataset
    try:
        dataset = pd.read_csv(arquivo)
    except FileNotFoundError:
        print(f"Erro: O arquivo {arquivo} não foi encontrado. Pulando para o próximo.")
        continue # Pula para o próximo arquivo se este não for encontrado

    # Selecionar colunas numéricas e a coluna de classe
    cols_num = dataset.select_dtypes(include=tipos_numericos)
    colunas_numericas = list(cols_num.columns)

    # Verificar se a coluna 'classe' existe e se há colunas numéricas
    if 'classe' not in dataset.columns:
        print(f"Aviso: O arquivo {arquivo} não contém a coluna 'classe'. Pulando para o próximo.")
        continue
    if not colunas_numericas:
        print(f"Aviso: O arquivo {arquivo} não contém colunas numéricas dos tipos especificados. Pulando para o próximo.")
        continue

    # Separate features (x) and target (y)
    x = dataset[colunas_numericas]
    y = dataset['classe']

    ## K-Fold Cross-Validation
    kf = KFold(n_splits=10, random_state=1, shuffle=True)
    gnb_kfold = GaussianNB()

    # Perform cross-validation and get accuracy scores for each fold
    scores_kfold = cross_val_score(gnb_kfold, x, y, scoring = 'accuracy', cv = kf)

    # Calculate the mean accuracy across the folds
    acuracia_kfold_mean = scores_kfold.mean()

    print(f"Acurácia K-Fold (média) para {arquivo}: {acuracia_kfold_mean:.3f}")

    # Store the K-Fold results
    resultados_kfold.append({
        'Base de Dados': arquivo,
        'Acurácia (K-Fold Média)': acuracia_kfold_mean
    })


# Save the results to a CSV file
df_resultados_kfold = pd.DataFrame(resultados_kfold)
df_resultados_kfold.to_csv('resultados_naive_bayes_kfold.csv', index=False)

print("Processamento concluído. Resultados salvos em 'resultados_naive_bayes_kfold.csv'")

Processando a base: VGG16_256_AVG.csv
Acurácia K-Fold (média) para VGG16_256_AVG.csv: 0.951
Processando a base: VGG16_256_MAX.csv
Acurácia K-Fold (média) para VGG16_256_MAX.csv: 0.972
Processando a base: VGG19_128_AVG.csv
Acurácia K-Fold (média) para VGG19_128_AVG.csv: 0.912
Processando a base: VGG19_128_MAX.csv
Acurácia K-Fold (média) para VGG19_128_MAX.csv: 0.940
Processando a base: VGG19_256_AVG.csv
Acurácia K-Fold (média) para VGG19_256_AVG.csv: 0.957
Processando a base: VGG19_256_MAX.csv
Acurácia K-Fold (média) para VGG19_256_MAX.csv: 0.975
Processando a base: VGG16_256_AVG_PCA.csv
Acurácia K-Fold (média) para VGG16_256_AVG_PCA.csv: 0.964
Processando a base: VGG16_256_MAX_PCA.csv
Acurácia K-Fold (média) para VGG16_256_MAX_PCA.csv: 0.974
Processando a base: VGG19_128_AVG_PCA.csv
Acurácia K-Fold (média) para VGG19_128_AVG_PCA.csv: 0.916
Processando a base: VGG19_128_MAX_PCA.csv
Acurácia K-Fold (média) para VGG19_128_MAX_PCA.csv: 0.936
Processando a base: VGG19_256_AVG_PCA.csv
Acurác